In [1]:
import telebot
from telebot import types
import requests
import os

bot = telebot.TeleBot(open(os.path.expanduser('~/.mybottoken')).read().strip())

stop = False

def update_dog():
    global dog
    r = requests.get('https://api.thedogapi.com/v1/images/search')
    dog = r.json()[0]['url']
    
    

def update_cat():
    global cat
    r = requests.get('https://api.thecatapi.com/v1/images/search')
    cat = r.json()[0]['url']
    
def stop():
    return stop == True


@bot.message_handler(commands=['start'])
def welcome(message):
    markup = types.InlineKeyboardMarkup()
    mrk = types.InlineKeyboardButton(text='Круто! Присылай!', callback_data='catalog')
    mrk1 = types.InlineKeyboardButton(text='Нет, спасибо!', callback_data='bye')
    markup.add(mrk,mrk1)
    bot.send_message(message.chat.id, 'Привет! Чтобы поднять тебе настроение, я могу присылать тебе фото кошечек или собачек каждый день.', reply_markup=markup)
    
    
@bot.message_handler(commands=['unsubscribe'])
def farewell(message):
    stop()
    bot.send_message(message.chat.id, 'Я больше не буду присылать тебе фото. Пока!')
    

@bot.callback_query_handler(func=lambda call: True)
def answer(call): 
    if call.data == 'catalog':
        markup1 = types.InlineKeyboardMarkup()
        mar = types.InlineKeyboardButton(text='Кошечки', callback_data='cats')
        mar2 = types.InlineKeyboardButton(text='Собачки', callback_data='dogs')
        markup1.add(mar,mar2)
        bot.send_message(call.message.chat.id,'Кого присылать?',reply_markup=markup1)
    elif call.data == 'bye':
        bot.send_message(call.message.chat.id, 'Что ж, очень жаль. Пока!', reply_markup=None)
    elif call.data == 'cats':
        bot.send_message(call.message.chat.id, 'Я буду присылать тебе кошечек. Чтобы отписаться, введи команду /unsubscribe', reply_markup=None)
        while stop == False:
            update_cat()
            bot.send_message(call.message.chat.id, cat, reply_markup=None)
            time.sleep(86400)
    elif call.data == 'dogs':
        bot.send_message(call.message.chat.id, 'Я буду присылать тебе собачек. Чтобы отписаться, введи команду /unsubscribe', reply_markup=None)
        while stop == False:
            update_dog()
            bot.send_message(call.message.chat.id, dog, reply_markup=None)
            time.sleep(86400)


bot.polling()

2021-02-22 20:40:57,590 (__init__.py:515 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
